# Imports 

In [ ]:
from aptos_sdk.account import Account
from aptos_sdk.bcs import Serializer
from aptos_sdk.client import RestClient, FaucetClient
from aptos_sdk.transactions import (
    EntryFunction,
    ModuleId,
    TransactionArgument as TxArg,
    TransactionPayload as TxPayload)
from aptos_sdk.type_tag import StructTag
from typing import Any, Dict

# Account via keyfile

In [ ]:
# Relative path to hot keyfile for testing only.
# File should contain hex key only.
key_file = '../../.secrets/2f4418ed678986669b22ca40c66\
4aa04707cacc5a326b47f0b4d540dff05e4c1.key'
with open(key_file) as f: # Open file.
    key = f.readline().rstrip() # Get key.
# Get account from key.
account = Account.load_key(key)
# Print account address in hex.
account.address().hex()

# REST Client 

In [ ]:
# Declare base URL for testnet REST API.
client_url = 'https://fullnode.testnet.aptoslabs.com/v1'
# Declare base URL for faucet.
faucet_url = 'https://faucet.testnet.aptoslabs.com'
# Initialize client.
client = RestClient(client_url)
# Initialize faucet.
faucet = FaucetClient(faucet_url, client)
# Get account balance
client.account_balance(account.address())

In [ ]:
# Fund from the faucet
faucet.fund_account(account.address().hex(), 1000000)

In [ ]:
# Get account balance
client.account_balance(account.address())

In [ ]:
def execute(function: str, args: list) -> str:
    """Call entry function with args, returning tx hash
    
    args should be of format
    [[4, Serializer.u64], [2, Serializer.u128]]
    """
    # Construct entry function payload.
    payload = EntryFunction.natural(
        str(module_id),
        function,
        [],
        [TxArg(a[0], a[1]) for a in args])
    # Generate a signed transaction from the payload.
    signed_tx = client.create_single_signer_bcs_transaction(
        account, TxPayload(payload))
    # Submit signed transaction, returning transaction ID.
    return client.submit_bcs_transaction(signed_tx)

In [ ]:
def get_tx_json(hash: str) -> Dict[str, Any]:
    """Query a transaction by hash, returning JSON data"""
    while(True):
        response = client.client.get(
            f'{client.base_url}/transactions/by_hash/{hash}')
        # Assert successful response.
        assert response.status_code == 200, hash
        # If transaction has cleared as user tx:
        if response.json()['type'] == 'user_transaction':
            # Return its JSON data
            return response.json()
        # Otherwise try again

In [ ]:
def print_tx_url(version: str):
    """Print URL to transaction view on explorer"""
    explorer = 'https://aptos-explorer.netlify.app'
    print(f'{explorer}/txn/{version}')

In [ ]:
def tx_diagnostics(tx_json: Dict[str, Any]):
    """Print gas used or link to failed tx"""
    if tx_json['success'] == True:
        print(f'Gas used: {tx_json["gas_used"]}')
    else:
        print_tx_url(tx_json['version'])

# Crit-queue interface

In [ ]:
def get_access_key(insertion_key: int,
                   insertion_count: int) -> int:
    """Return access key for insertion key/count"""
    return insertion_key << 64 | insertion_count

In [ ]:
# Declare module name.
module = 'critqueue_benchmark'
# Get module ID.
module_id = ModuleId(account.address(), module)
# Get CritQueueStore struct tag.
struct_tag = StructTag(account.address(),
                       module,
                      'CritQueueStore',
                       [])
# Check that account has one.
client.account_resource(account.address(),
                        struct_tag.__str__())

In [ ]:
def insert(insertion_key: int,
           insertion_value: int) -> str:
    """Insert given key-value insertion pair,
    returning tx ID"""
    return execute('insert', [
        [insertion_key, Serializer.u64],
        [insertion_value, Serializer.u64]
    ])

In [ ]:
def remove(insertion_key: int,
           insertion_count: int,
           insertion_value_expected: int) -> str:
    """Remove corresponding key-value insertion pair,
    returning tx ID"""
    access_key = get_access_key(
        insertion_key, insertion_count)
    return execute('remove', [
        [access_key, Serializer.u128],
        [insertion_value_expected, Serializer.u64]
    ])

In [ ]:
def dequeue(insertion_value_expected: int) -> str:
    """Dequeue, checking insertion value and
    returning tx ID"""
    return execute('dequeue', [
        [insertion_value_expected, Serializer.u64]
    ])

In [ ]:
def dequeue_all() -> str:
    """Dequeue all, returning tx ID"""
    return execute('dequeue_all', [])

In [ ]:
def reset() -> str:
    """Reset crit-queue, returning tx ID"""
    return execute('reset', [])

In [ ]:
# Insert key-value pair.
tx_diagnostics(get_tx_json(insert(10, 1)))

In [ ]:
# Try removing key-value pair.
tx_diagnostics(get_tx_json(remove(10, 0, 1)))

In [ ]:
# Insert several key-value pairs.
for i in range(10):
    # Wait until successful.
    tx_diagnostics(get_tx_json(insert(10, 1)))
# Dequeue.
tx_diagnostics(get_tx_json(dequeue(1)))

In [ ]:
# Dequeue all
tx_diagnostics(get_tx_json(dequeue_all()))

In [ ]:
# Reset
tx_diagnostics(get_tx_json(reset()))